# Load files

In [ ]:
filename_train = '../input/Kannada-MNIST/train.csv'
filename_test  = '../input/Kannada-MNIST/test.csv'

In [ ]:
import pandas as pd

df_train = pd.read_csv(filename_train)
df_test  = pd.read_csv(filename_test )

# Reshape data

In [ ]:
import tensorflow as tf

# training data
X_train = df_train.values[:, 1:].reshape((-1, 28, 28, 1)) / 255.
y_train = tf.keras.utils.to_categorical(df_train.values[:, 0], 10)
# test data
X_test = df_test.values[:, 1:].reshape((-1, 28, 28, 1)) / 255.

# Modeling

In [ ]:
input = tf.keras.layers.Input(shape=(28, 28, 1))

x = input
x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)

output = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.models.Model(input, output)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# Learning

In [ ]:
image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.1)
image_data_generator.fit(X_train)

In [ ]:
epochs = 100
batch_size = 64

# コールバック関数を定義
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
]

train_generator = image_data_generator.flow(X_train, y_train, batch_size=batch_size, subset='training')
valid_generator = image_data_generator.flow(X_train, y_train, batch_size=batch_size, subset='validation')

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // batch_size, 
                    validation_data=valid_generator,
                    validation_steps=valid_generator.n // batch_size,
                    epochs=epochs,
                    callbacks=callbacks)

# Prediction

In [ ]:
y_pred = model.predict(X_test).argmax(axis=-1)

In [ ]:
import numpy as np

y_pred = pd.DataFrame([np.arange(len(y_pred)), y_pred], index=['id', 'label']).T
y_pred

In [ ]:
filename_output = './submission.csv'
y_pred.to_csv(filename_output, index=False)